In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
from keras.datasets import imdb
idx = imdb.get_word_index()
idx

2113536/2343108 [==========================>...] - ETA: 0s

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [4]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [5]:
idx2word = {v: k for k, v in idx.iteritems()}

In [6]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

64692224/65552540 [============================>.] - ETA: 0s

In [10]:
len(x_train)

25000

In [11]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [13]:
idx2word[23022]

'bromwell'

In [14]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [15]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [19]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [21]:
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

In [22]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [23]:
trn.shape

(25000, 500)

In [24]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [25]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 16000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           1600100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [26]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 4s - loss: 0.4638 - acc: 0.7522 - val_loss: 0.2970 - val_acc: 0.8726
Epoch 2/2
25000/25000 [==============================] - 3s - loss: 0.2018 - acc: 0.9244 - val_loss: 0.3039 - val_acc: 0.8739


In [28]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 12s - loss: 0.4708 - acc: 0.7476 - val_loss: 0.2821 - val_acc: 0.8843
Epoch 2/4
25000/25000 [==============================] - 12s - loss: 0.2901 - acc: 0.8848 - val_loss: 0.2632 - val_acc: 0.8925
Epoch 3/4
25000/25000 [==============================] - 11s - loss: 0.2582 - acc: 0.9006 - val_loss: 0.2623 - val_acc: 0.8898
Epoch 4/4
25000/25000 [==============================] - 12s - loss: 0.2392 - acc: 0.9070 - val_loss: 0.2572 - val_acc: 0.8941


In [30]:
model_path = '/home/ubuntu/IMDB/imdb-sentiment/model_path'
%mkdir -p $model_path

In [31]:
conv1.save_weights(model_path + 'conv1.h5')
conv1.load_weights(model_path + 'conv1.h5')

In [32]:
def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('/home/ubuntu/IMDB/imdb-sentiment/glove')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [33]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [34]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.50d'))

79618048/80107627 [============================>.] - ETA: 0sUntaring file...


In [35]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [36]:
emb = create_emb()

In [37]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [38]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [39]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 12s - loss: 0.6310 - acc: 0.6312 - val_loss: 0.5371 - val_acc: 0.7473
Epoch 2/2
25000/25000 [==============================] - 12s - loss: 0.5122 - acc: 0.7569 - val_loss: 0.4565 - val_acc: 0.8070


In [40]:
model.layers[0].trainable=True

In [41]:
model.optimizer.lr=1e-4

In [42]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 12s - loss: 0.4781 - acc: 0.7766 - val_loss: 0.4432 - val_acc: 0.8119


In [43]:
from keras.layers import Merge

In [124]:
graph_in = Input ((vocab_size, 50))
convs = [ ] 
for fsz in range (3, 6): 
    x = Convolution1D(50, fsz, border_mode='same', activation="relu")(graph_in)
    x = Dropout(0.5)(x)
    x = Convolution1D(50, fsz, border_mode='same', activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Convolution1D(50, fsz, border_mode='same', activation="relu")(x)
    x = Dropout(0.5)(x)
    x = merge([x, graph_in], mode='sum')
    
    x = Convolution1D(50, fsz, border_mode='same', activation="relu")(graph_in)
    x = Dropout(0.6)(x)
    x = Convolution1D(50, fsz, border_mode='same', activation="relu")(x)
    x = Dropout(0.6)(x)
    x = Convolution1D(50, fsz, border_mode='same', activation="relu")(x)
    x = Dropout(0.6)(x)
    x = merge([x, graph_in], mode='sum')
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode="concat")(convs) 
graph = Model(graph_in, out) 

In [125]:
emb = create_emb()

In [126]:
model = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.6),
    graph,
    Dropout (0.7),
    Dense (100, activation="relu"),
    Dropout (0.8),
    Dense (1, activation='sigmoid')
    ])

In [129]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [130]:
model.lr=0.01
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=10, batch_size=64)
model.lr = 0.005
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=10, batch_size=64)
model.lr=0.001
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=10, batch_size=64)


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 60s - loss: 0.7068 - acc: 0.5066 - val_loss: 0.6922 - val_acc: 0.5278
Epoch 2/10
25000/25000 [==============================] - 60s - loss: 0.6929 - acc: 0.5075 - val_loss: 0.6896 - val_acc: 0.5521
Epoch 3/10
25000/25000 [==============================] - 60s - loss: 0.6875 - acc: 0.5370 - val_loss: 0.6795 - val_acc: 0.6494
Epoch 4/10
25000/25000 [==============================] - 60s - loss: 0.6736 - acc: 0.5634 - val_loss: 0.6231 - val_acc: 0.7434
Epoch 5/10
25000/25000 [==============================] - 60s - loss: 0.5796 - acc: 0.6720 - val_loss: 0.4682 - val_acc: 0.8591
Epoch 6/10
25000/25000 [==============================] - 60s - loss: 0.4512 - acc: 0.7936 - val_loss: 0.4388 - val_acc: 0.8772
Epoch 7/10
25000/25000 [==============================] - 60s - loss: 0.3997 - acc: 0.8317 - val_loss: 0.3585 - val_acc: 0.8718
Epoch 8/10
25000/25000 [==============================

In [105]:
model.lr = 0.005
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=10, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 53s - loss: 0.1808 - acc: 0.9292 - val_loss: 0.2566 - val_acc: 0.8987
Epoch 2/10
25000/25000 [==============================] - 53s - loss: 0.1749 - acc: 0.9320 - val_loss: 0.2701 - val_acc: 0.8950
Epoch 3/10
25000/25000 [==============================] - 53s - loss: 0.1690 - acc: 0.9339 - val_loss: 0.2601 - val_acc: 0.9028
Epoch 4/10
25000/25000 [==============================] - 53s - loss: 0.1595 - acc: 0.9396 - val_loss: 0.2651 - val_acc: 0.8979
Epoch 5/10
25000/25000 [==============================] - 53s - loss: 0.1522 - acc: 0.9413 - val_loss: 0.2543 - val_acc: 0.9011
Epoch 6/10
25000/25000 [==============================] - 53s - loss: 0.1446 - acc: 0.9438 - val_loss: 0.2652 - val_acc: 0.9010
Epoch 7/10
25000/25000 [==============================] - 53s - loss: 0.1404 - acc: 0.9480 - val_loss: 0.2867 - val_acc: 0.8976
Epoch 8/10
25000/25000 [==============================